# Remote Backend Compiler - RBC
*by Pearu Peterson*

## Statement of the problem

*Compile and run user-defined functions in a language-agnostic JIT enabled program semi-remotely.*

### Semi-remote compilation

- *compiler frontend*: user-written source code is parsed and transformed to LLVM IR in client program
- the client program sends the LLVM IR over network to a server program
- *compiler backend*: the server program compiles the LLVM IR to machine code (that can be executed on the server's CPU or GPU devices)

### Constraints

- A user-defined function is defined in Python or C/C++
  (or any other language with LLVM tools)
- Client host runs on a 32- or 64-bit OS: Linux, MacOSX, Windows
- Server host runs on a 64-bit Linux

## Solution

- *Remote Backend Compiler - RBC*: https://github.com/xnd-project/rbc
- A Python-to-Python prototype uses Numba for LLVM IR generation and Numba llvmlite for machine code compilation
- A Python-to-SQL application uses Numba for LLVM IR generation and OmniSciDB for machine code compilation
  * OmniSciDB is a GPU enabled SQL database server
  * OmniSciDB uses LLVM Compiler C++ library for JIT compilation
  * https://www.omnisci.com/

# Demo of the prototype below

In [1]:
#NBVAL_IGNORE_OUTPUT
from rbc import RemoteJIT
rjit = RemoteJIT(host='localhost', port=7890)
rjit.start_server(background=True)

staring rpc.thrift server: /home/guilhermel/.conda/envs/rbc/lib/python3.8/site-packages/rbc/remotejit.thrift

In [25]:
# Client-server connector, RBC jit-decorator
rjit = RemoteJIT(host='localhost', port=7890)

In [26]:
# A user-defined function
@rjit('int64(int64, int64)')
def foo(a, b):
    return a + b

In [27]:
# Generate LLVM IR, useful for debugging
#print(foo)

In [28]:
# Triggers:
#  - local frontend compile,
#  - remote backend compile,
#  - and remote call
foo(1, 3)

4

In [29]:
foo(2, 4)  # reuses remote compile result and triggers remote call

6

In [30]:
try:
    foo(1.2, 3.4)
except Exception as msg:
    print(msg)

found no matching function type to given argument types `float64, float64`. Available function types: int64(int64, int64)


In [31]:
# update with new signatures
foo.signature('double(double, double)')
foo.signature('complex128(complex128, complex128)');

In [32]:
foo(1, 3.4j)  # triggers remote compile and call

(1+3.4j)

In [33]:
foo(1.2, 3)  # triggers remote compile and call

4.2

In [34]:
rjit.stop_server()